In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
ng_train = fetch_20newsgroups(subset = 'train',shuffle=True)
ng_test = fetch_20newsgroups(subset = 'test',shuffle=True)

In [ ]:
print(list(ng_train.target_names))

In [ ]:
ng_train.data[:3]

In [ ]:
print(ng_train.filenames.shape,ng_train.target.shape)

In [ ]:
!pip install gensim

In [ ]:
!pip install nltk

In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem import *
import numpy as np
np.random.seed(400)

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
WordNetLemmatizer().lemmatize('runs')

In [ ]:
import pandas as pd
stemmer = SnowballStemmer("english")
original_words =['caresses', 'flies', 'dies', 'mules', 'dined', 'died', 'agreed', 'owned', 'humled', 'sized', 'meeting', 'stating', 'seizing', 'itemization', 'sensational', 'traditional', 'reference', 'colonizer', 'plotted']

singles = [WordNetLemmatizer().lemmatize(plural,pos='v') for plural in original_words]

pd.DataFrame(data={'Original Words' : original_words, 'Lemma':singles})


In [ ]:
!pip install gensim

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lematize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lematize_stemming(token))
            return result
            

In [ ]:
#preview a documnet before preprocessing
#document num = 50
doc_sample = 'this disk has failed many times. I would like to get it replaced.'

print("Original Document:")
words=[]
for word in doc_sample.split(' '):
    words.append(word)
    print(words)
    print("\n\nTokenized and lemmatized document: ")
    print(preprocess(doc_sample))

In [ ]:
processed_docs = []

for doc in ng_train.data:
    processed_docs.append(preprocess(doc))

In [ ]:
len(processed_docs)

In [ ]:
print(processed_docs[:2])

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
print(dictionary)

In [ ]:
count=0
for k,v in dictionary.iteritems():
    print(k, v)
    count+=1
    if count>10:
        break

In [ ]:
dictioanty.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
bow_corpus[:5]

In [ ]:
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("word{}(\"{}\")appears {} time. ".format(bow_doc_x[i][0],
                                                  dictionary[bow_doc_x[i][0]],
                                                  bow_doc_x[i][1]))

In [ ]:
lda_model = gensim.models.ldamulticore(bow_corpus,
                                      num_topics = 2,
                                      id2word = dictionary,
                                      passes = 10,
                                      workers = 2)

In [ ]:
for idx,topic in lda_model.print_topics():
    print("Topic: {} \nWords: {}",format(idx,topic))
    print("\n")

In [ ]:
num = 15
unseen_document = ng_test.data[num]
print(unseen_document)

In [ ]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector]):
    print("Score: {}\t Topic: {}".format(score,lda_model.print_topic(index,10)))